In [5]:
import pygame
import numpy as np
from math import sin, cos, pi


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


ModuleNotFoundError: No module named 'numpy'

In [4]:
# Initialize Pygame
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height), pygame.RESIZABLE)
pygame.display.set_caption("4D Polyhedra Visualizer")


NameError: name 'pygame' is not defined

In [ ]:
# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (100, 100, 100)
RED = (255, 0, 0)
BLUE = (50, 100, 255)
GREEN = (50, 200, 50)


In [ ]:
# Shape Generation Functions
def generate_hypercube():
    # Generate vertices and edges for a 4D hypercube (tesseract)
    vertices = np.array([
        [x, y, z, w] for x in [-1, 1] for y in [-1, 1]  
        for z in [-1, 1] for w in [-1, 1]
    ], dtype=float)
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            diff = np.sum(vertices[i] != vertices[j])
            if diff == 1:
                edges.append((i, j))
    return vertices, edges, "Hypercube (Tesseract)"


In [ ]:
def generate_5cell():
    # Generate vertices and edges for a 5-cell (4D simplex)
    r = 1.5  # Scale factor for better visualization
    vertices = np.array([
        [0, 0, 0, -r/4],  # Base vertex
        [r, 0, 0, r/4],    # Remaining vertices in a tetrahedral arrangement
        [-r/3, r*0.94, 0, r/4],
        [-r/3, -r*0.47, r*0.82, r/4],
        [-r/3, -r*0.47, -r*0.82, r/4],
    ], dtype=float)
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            edges.append((i, j))
    return vertices, edges, "5-cell (4D Simplex)"


In [ ]:
def generate_16cell():
    # Generate vertices and edges for a 16-cell (4D hexadecachoron)
    r = 1.0
    vertices = np.array([
        [r, 0, 0, 0],
        [-r, 0, 0, 0],
        [0, r, 0, 0],
        [0, -r, 0, 0],
        [0, 0, r, 0],
        [0, 0, -r, 0],
        [0, 0, 0, r],
        [0, 0, 0, -r],
    ], dtype=float)
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            edges.append((i, j))
    return vertices, edges, "16-cell (Hexadecachoron)"


In [ ]:
def generate_24cell():
    # Generate vertices and edges for a 24-cell
    vertices = []
    r = 1.0
    for x in [-r, r]:
        for y in [-r, r]:
            vertices.append([x, y, 0, 0])
            vertices.append([x, 0, y, 0])
            vertices.append([x, 0, 0, y])
            vertices.append([0, x, y, 0])
            vertices.append([0, x, 0, y])
            vertices.append([0, 0, x, y])
    vertices = np.array(vertices, dtype=float)
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            dist = np.linalg.norm(vertices[i] - vertices[j])
            if abs(dist - np.sqrt(2)) < 0.01:
                edges.append((i, j))
    return vertices, edges, "24-cell"


In [ ]:
def generate_8cell():
    # Generate vertices and edges for an 8-cell (4D octahedron)
    r = 1.5
    vertices = np.array([
        [r, 0, 0, 0],
        [-r, 0, 0, 0],
        [0, r, 0, 0],
        [0, -r, 0, 0],
        [0, 0, r, 0],
        [0, 0, -r, 0],
        [0, 0, 0, r],
        [0, 0, 0, -r],
    ], dtype=float)
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            same_zeros = sum(1 for a, b in zip(vertices[i], vertices[j]) if a == 0 and b == 0)
            if same_zeros == 3:
                edges.append((i, j))
    return vertices, edges, "8-cell (4D Octahedron)"


In [ ]:
# Define all available shapes
shape_generators = [
    generate_hypercube,
    generate_5cell,
    generate_16cell,
    generate_24cell,
    generate_8cell
]


In [ ]:
# Main game loop
running = True
angles = [0, 0, 0, 0, 0, 0]  # XY, XZ, XW, YZ, YW, ZW
rotation_speed = 0.02
zoom_factor = 1.0  # Initialize zoom factor


In [ ]:
# Create slider for rotation speed
speed_slider = Slider(width // 2 - 100, height - 50, 200, 20, 0.0, 0.05, rotation_speed)


In [ ]:
# Function to change shape
def change_shape():
    global current_shape_index, vertices_4d, edges, shape_name
    current_shape_index = (current_shape_index + 1) % len(shape_generators)
    vertices_4d, edges, shape_name = shape_generators[current_shape_index]()
    return True


In [ ]:
# Main loop for Pygame
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_SPACE:
                change_shape()
        elif event.type == pygame.VIDEORESIZE:
            width, height = event.size
            screen = pygame.display.set_mode((width, height), pygame.RESIZABLE)
            scale = calculate_scale(width, height)
        elif event.type == pygame.MOUSEWHEEL:
            zoom_factor *= 1.1 if event.y > 0 else 0.9
            zoom_factor = max(0.1, min(zoom_factor, 10.0))
            scale = calculate_scale(width, height)


In [ ]:
    # Update rotation speed from slider
    rotation_speed = speed_slider.value
    angles = [angle + rotation_speed for angle in angles]


In [ ]:
    # Rotate and project
    rotated_vertices = rotate4d(vertices_4d, *angles)
    projected_3d = project_4d_to_3d(rotated_vertices)
    projected_2d = project_3d_to_2d(projected_3d)


In [ ]:
    # Scale and center the 2D points
    screen_points = (projected_2d * scale + np.array([width/2, height/2])).astype(int)


In [ ]:
    # Draw
    screen.fill(BLACK)
    font = pygame.font.SysFont(None, 32)
    title = font.render(shape_name, True, GREEN)
    screen.blit(title, (width//2 - title.get_width()//2, 10))


In [ ]:
    for i, point in enumerate(screen_points):
        pygame.draw.circle(screen, WHITE, point, 3)


In [ ]:
    for edge in edges:
        start = screen_points[edge[0]]
        end = screen_points[edge[1]]
        pygame.draw.line(screen, WHITE, start, end, 1)


In [ ]:
    speed_slider.draw(screen)
    change_shape_button.draw(screen)
    font = pygame.font.SysFont(None, 16)
    instructions = font.render("Press SPACE to change shape, scroll wheel to zoom", True, WHITE)
    screen.blit(instructions, (10, 40))


In [ ]:
    pygame.display.flip()
    clock.tick(60)


In [ ]:
pygame.quit()
